In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/sample_data/california_housing_test.csv')

In [ ]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0


In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a8c46798ce7b4d78b4796fe0c5933509cb1a8b92abcad8ee7f948c534d78488c
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

spark = SparkSession.builder.appName('Myapp').getOrCreate()

sdf = spark.createDataFrame(df)

#**Creating Data Frame**

###**Method 1: Using schema**

In [ ]:
schema = StructType([
    StructField('Name', StringType()),
    StructField('Age', IntegerType()),
    StructField('Salary', FloatType())
])

dic = [{'Name': 'Vasant',
       'Age': 25,
       'Salary': 140000.00}]
new_df = spark.createDataFrame(dic, schema)
new_df.show()

+------+---+--------+
|  Name|Age|  Salary|
+------+---+--------+
|Vasant| 25|140000.0|
+------+---+--------+



###**Method 2: Using Row**

In [ ]:
row = [Row('Vasant', 25, 14000.00),
       Row('Mohit', 45, 4500000.0)]

new_df2 = spark.createDataFrame(row)

new_df2 = new_df2.withColumnRenamed('_1','Name')
new_df2 = new_df2.withColumnRenamed('_2','Age')
new_df2 = new_df2.withColumnRenamed('_3','Salary')

new_df2.show()

+------+---+---------+
|  Name|Age|   Salary|
+------+---+---------+
|Vasant| 25|  14000.0|
| Mohit| 45|4500000.0|
+------+---+---------+



###**Method 3: Using RDD**

In [ ]:
row = [Row('Vasant', 25, 14000.00),
       Row('Mohit', 45, 4500000.0)]

rdd = spark.sparkContext.parallelize(row)
new_df3 = spark.createDataFrame(rdd)

new_df3.show()

+------+---+---------+
|    _1| _2|       _3|
+------+---+---------+
|Vasant| 25|  14000.0|
| Mohit| 45|4500000.0|
+------+---+---------+



#**Data Pre-processing**

In [ ]:
df = spark.read.csv('/content/sample_data/california_housing_test.csv',
                    inferSchema = True,
                    header = True)
print('Number of rows:', df.count())
df.show()

Number of rows: 3000
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -122.05|   37.37|              27.0|     3885.0|         661.0|    1537.0|     606.0|       6.6085|          344700.0|
|   -118.3|   34.26|              43.0|     1510.0|         310.0|     809.0|     277.0|        3.599|          176500.0|
|  -117.81|   33.78|              27.0|     3589.0|         507.0|    1484.0|     495.0|       5.7934|          270500.0|
|  -118.36|   33.82|              28.0|       67.0|          15.0|      49.0|      11.0|       6.1359|          330000.0|
|  -119.67|   36.33|              19.0|     1241.0|         244.0|     850.0|     237.0|       2.9375|           81700.0|
|  

In [ ]:
# Filtering data
print(df.filter("housing_median_age > 25").count())

print(df.filter(df.housing_median_age > 25).count())

1772
1772


### Data Partitioning

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, concat

df = df.withColumn("ID", row_number().over(Window().orderBy('median_house_value'))).select('ID', *[i for i in df.columns if i!='ID'])
df.show()

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
| ID|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  1|  -117.16|   32.71|              52.0|      845.0|         451.0|    1230.0|     375.0|       1.0918|           22500.0|
|  2|  -123.08|    40.4|              10.0|      365.0|         102.0|     140.0|      49.0|       1.7969|           37500.0|
|  3|  -119.97|   36.44|              18.0|     1128.0|         237.0|     772.0|     220.0|       2.1771|           39200.0|
|  4|  -117.65|    35.0|              36.0|     1184.0|         316.0|     672.0|     241.0|       1.9107|           39800.0|
|  5|  -122.06|   37.39|              22.0|     1236.0|         290.0|     413.0|     274.0|       3.6875|           4

In [ ]:
# Hash Partitioning:
# Distributes data across partitions using a hash of the 'ID' column.
# Useful for parallel processing, joins, and aggregations on the same key.

df.repartition(1000, 'ID').write.option('header', True).mode('overwrite').csv('HashPartitioning')

df.repartition(1000, 'ID').rdd.glom().collect()

#####**Output is Truncated bcz of the huge size**

In [ ]:
# Range Partitioning:
# Partitions data based on sorted ranges of 'ID'.
# Useful for range queries, ordered processing, and avoiding data skew.

df.repartitionByRange(1000, 'ID').write.option('header', True).mode('overwrite').csv('partitionByRange')

df.repartitionByRange(1000, 'ID').rdd.glom().collect()

#####**Output is Truncated bcz of the huge size**

In [ ]:
# PartitionBy (Directory-based partitioning):
# Writes data into directory structure based on column values.
# Useful for query pruning and faster reads in analytics workloads.

df.write.mode('overwrite').option('header', True).partitionBy('median_income','median_house_value').csv('partitionBy')

In [ ]:
df.write.option('header', True).mode('overwrite').csv('Original')

In [ ]:
# Hash partitioning fetch

in_df = spark.read.option('header', True).csv('/content/HashPartitioning')
in_df.count()

3000

In [ ]:
# PartiotionByRange fetch
in_df = spark.read.option('header', True).csv('/content/partitionByRange')
in_df.count()

3000

In [ ]:
# PartitionBy fetch
in_df = spark.read.option('header', True).csv('/content/partitionBy')
in_df.count()

3000

In [ ]:
in_df.show()

+----+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  ID|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+----+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|2876|  -118.06|   34.15|              37.0|     1980.0|         226.0|     697.0|     226.0|      15.0001|          500001.0|
|2881|  -117.85|   33.62|              13.0|     5192.0|         658.0|    1865.0|     662.0|      15.0001|          500001.0|
|2931|  -118.37|    34.1|              37.0|      407.0|          67.0|     100.0|      47.0|      15.0001|          500001.0|
|2935|  -118.42|   34.09|              40.0|     3552.0|         392.0|    1024.0|     370.0|      15.0001|          500001.0|
|2939|   -118.1|   34.13|              47.0|     2234.0|         276.0|     749.0|     260.0|      15.0001|    

In [ ]:
from pyspark.sql.functions import *
ratio = udf(lambda total_rooms, total_bedrooms: total_rooms/total_bedrooms, FloatType())

df.withColumn('Construct_year', 2023 - df.housing_median_age).show()

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+--------------+
| ID|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|Construct_year|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+--------------+
|  1|  -117.16|   32.71|              52.0|      845.0|         451.0|    1230.0|     375.0|       1.0918|           22500.0|        1971.0|
|  2|  -123.08|    40.4|              10.0|      365.0|         102.0|     140.0|      49.0|       1.7969|           37500.0|        2013.0|
|  3|  -119.97|   36.44|              18.0|     1128.0|         237.0|     772.0|     220.0|       2.1771|           39200.0|        2005.0|
|  4|  -117.65|    35.0|              36.0|     1184.0|         316.0|     672.0|     241.0|       1.9107|           39800.0|        1987.0|
|  5|  -122.0

In [ ]:
from functools import reduce

updated_df = (reduce(lambda data, column: data.withColumn(column, sqrt(col(column))), df.columns, df))

updated_df.show()

+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                ID|longitude|          latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|        households|     median_income|median_house_value|
+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|               1.0|      NaN| 5.719265687131522| 7.211102550927978|29.068883707497267|21.236760581595302| 35.07135583350036|19.364916731037084| 1.044892338951722|             150.0|
|1.4142135623730951|      NaN| 6.356099432828281|3.1622776601683795|  19.1049731745428|10.099504938362077|11.832159566199232|               7.0|1.3404849868610986|193.64916731037084|
|1.7320508075688772|      NaN| 6.036555309114628| 4.242640687119285| 33.5857112474933

In [ ]:
#same operation with for loop
up_df = df.select('*')
for c in df.columns:
  up_df = up_df.withColumn(c, sqrt(col(c).cast(IntegerType())))

up_df.show()

+------------------+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                ID|longitude|         latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|        households|     median_income|median_house_value|
+------------------+---------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|               1.0|      NaN|5.656854249492381| 7.211102550927978|29.068883707497267|21.236760581595302| 35.07135583350036|19.364916731037084|               1.0|             150.0|
|1.4142135623730951|      NaN|6.324555320336759|3.1622776601683795|  19.1049731745428|10.099504938362077|11.832159566199232|               7.0|               1.0|193.64916731037084|
|1.7320508075688772|      NaN|              6.0| 4.242640687119285| 33.58571124749333|15.3

In [ ]:
new_df = df.select(*[sqrt(col(c)).alias(c) for c in df.columns])
new_df.show()

+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|                ID|longitude|          latitude|housing_median_age|       total_rooms|    total_bedrooms|        population|        households|     median_income|median_house_value|
+------------------+---------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+
|               1.0|      NaN| 5.719265687131522| 7.211102550927978|29.068883707497267|21.236760581595302| 35.07135583350036|19.364916731037084| 1.044892338951722|             150.0|
|1.4142135623730951|      NaN| 6.356099432828281|3.1622776601683795|  19.1049731745428|10.099504938362077|11.832159566199232|               7.0|1.3404849868610986|193.64916731037084|
|1.7320508075688772|      NaN| 6.036555309114628| 4.242640687119285| 33.5857112474933

In [ ]:
import numpy as np

lis_student = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
for i in range(2092):
  lis_student.append(lis_student[np.random.randint(0,7)])

label_df = udf(lambda ind: lis_student[ind-1], StringType())

df = df.withColumn('Random', label_df('ID'))

df.show()

+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------+
| ID|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|Random|
+---+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+------+
|  1|  -117.16|   32.71|              52.0|      845.0|         451.0|    1230.0|     375.0|       1.0918|           22500.0|     A|
|  2|  -123.08|    40.4|              10.0|      365.0|         102.0|     140.0|      49.0|       1.7969|           37500.0|     B|
|  3|  -119.97|   36.44|              18.0|     1128.0|         237.0|     772.0|     220.0|       2.1771|           39200.0|     C|
|  4|  -117.65|    35.0|              36.0|     1184.0|         316.0|     672.0|     241.0|       1.9107|           39800.0|     D|
|  5|  -122.06|   37.39|              22.0|     1236.0|         290.0

In [ ]:
df = df.drop('ID_2')
df = df.withColumn('ID_2', monotonically_increasing_id())
df.show()